# MATH50003 Numerical Analysis (2021–2022) Computer-based Exam

Instructions for uploading and downloading:

1. Rename the file to include your CID.
2. You have 30 mins to download the exam beginning at 3pm on 18 March.
2. You have 1 hour to complete the exam beginning at 3:30pm on 18 March.
3. Deadline is 5pm on 18 March to upload the completed Jupyter notebook (`.ipynb`) to Blackboard. 
Late submissions received before 7pm will be capped at 40%.
5. Once uploaded, re-download the file before the final submission time to confirm it is correct.
You are allowed to upload additional submissions but only last valid upload before 5pm will be used.
6. If uploading via Blackboard fails you may e-mail the UG Office: maths.exams@imperial.ac.uk

Instructions for the exam:

1. For each problem, replace the `# TODO` to complete the question.
The unit tests are provided to help you test your answers.
3. Problems are marked A/B/C to indicate difficulty ("A" being most difficult).
Partial credit will be awarded for reasonable attempts even if the tests
are not passed. A and B questions are worth 12 marks while C questions are worth 10 marks.
3. If you have technical queries please email s.olver@imperial.ac.uk. Any other queries
should be sent to the UG Office: maths.exams@imperial.ac.uk
4. You may use existing code from anywhere
but you are **REQUIRED** to cite the source if it is not part of the module material,
ideally by including a weblink in a comment. 
5. You **MUST NOT** ask for help online or
communicate with others within or outside the module.
Failure to follow these rules will be considered misconduct.



You should use the following packages:

In [1]:
using LinearAlgebra, Test

(Note `SetRounding` is not necessary.)

**WARNING** It may be necessary to restart the kernel if issues arise. Remember to reload the packages
when you do so.

## 1. Numbers

**Problem 1.1 (C)** 
Implement the function `issub` that determines whether a `Float16` is a sub-normal number.
DO NOT use the inbuilt routine `issubnormal`.

In [2]:
function issub(x::Float16)
    # TODO: return `true` if `x` is a sub-normal float. Otherwise return `false`
    ## SOLUTION
    σ = 15
    abs(x) < 2.0^(1-σ)
    ## END
end

issub (generic function with 1 method)

In [3]:
@test issub(Float16(0))
@test issub(nextfloat(Float16(0)))
@test issub(prevfloat(Float16(0)))
@test !issub(Float16(1))
@test !issub(reinterpret(Float16,0b0000010000000000))
@test issub(reinterpret(Float16,0b0000001111111111))

Test Passed
  Expression: issub(reinterpret(Float16, 0x03ff))

## 2. Differentiation

**Problem 2.1 (C)** Use second-order finite differences
with an appropriately chosen $h$ to approximate the second derivative of
$$
f(x) = \cos(x^2)
$$
at $x = 0.1$ to 5 digits accuracy.

In [4]:
function fd2(x)
    # TODO: implement a second-order finite-difference rule 
    # to approximate f''(x)
    # for f(x) = cos(x^2)
    # with step-size h chosen to get sufficient accuracy
    ## SOLUTION
    h = cbrt(eps())
    f = x -> cos(x^2)
    (f(x + h) - 2f(x) + f(x - h)) / h^2
    ## END
end

fd2 (generic function with 1 method)

In [5]:
@test abs(fd2(0.1) + 2*sin(0.1^2) + 4*0.1^2*cos(0.1^2)) ≤ 1E-5

Test Passed
  Expression: abs(fd2(0.1) + 2 * sin(0.1 ^ 2) + 4 * 0.1 ^ 2 * cos(0.1 ^ 2)) ≤ 1.0e-5
   Evaluated: 2.2397068676072163e-7 ≤ 1.0e-5

**Problem 2.2 (A)** Consider a 2nd order version of a dual number:
$$
a + b ϵ_1 + c ϵ_2
$$
such that
$$
\begin{align*}
ϵ_1^2 &= ϵ_2 \\
ϵ_2^2 &= ϵ_1 ϵ_2 =  0.
\end{align*}
$$
Complete the following implementation supporting `+` and `*` (and
assuming `a,b,c` are `Float64`). Hint: you may need to work out on paper
how to multiply `(s.a + s.b ϵ_1 + s.c ϵ_2)*(t.a + t.b ϵ_1 + t.c ϵ_2)` using the
relationship above.

In [6]:
import Base: *, +, ^
struct Dual2
    a::Float64
    b::Float64
    c::Float64
end

function +(s::Dual2, t::Dual2)
    ## TODO: Implement Dual2(...) + Dual2(...), returning a Dual2
    ## SOLUTION
    Dual2(s.a + t.a, s.b + t.b, s.c + t.c)
    ## END
end

function +(s::Dual2, c::Real)
    ## TODO: Implement Dual2(...) + c, returning a Dual2
    ## SOLUTION
    Dual2(s.a + c, s.b, s.c)
    ## END
end

function *(c::Number, s::Dual2)
    ## TODO: Implement c * Dual2(...), returning a Dual2
    ## SOLUTION
    Dual2(c * s.a, c * s.b, c * s.c)
    ## END
end

function *(s::Dual2, t::Dual2)
    ## TODO: Implement Dual2(...) * Dual2(...), returning a Dual2

    ## SOLUTION
    # we deduce (s.a + s.b*ϵ_1 + s.c*ϵ_2)*(t.a + t.b*ϵ_1 + t.c*ϵ_2) == 
    # s.a * t.a + (s.a*t.b + s.b*t.a)*ϵ_1 + (s.a*t.c + s.b*t.b + s.c*t.a)*ϵ_2
    Dual2(s.a * t.a, s.a * t.b + s.b * t.a, s.a * t.c + s.b * t.b + s.c * t.a)
    ## END
end

* (generic function with 406 methods)

In [7]:
f = x -> x*x*x + 2x + 1
x = 0.1
@test f(Dual2(x,1.,0.)) == Dual2(f(x), 3x^2+2, 6x / 2)

# This has computed the first and second derivatives as
# as f(x) + f'(x)*ϵ_1 + f''(x)/2*ϵ_2
# == (x^3 + x) + (3x^2+1)*ϵ_1 + 6x/2*ϵ_2

Test Passed
  Expression: f(Dual2(x, 1.0, 0.0)) == Dual2(f(x), 3 * x ^ 2 + 2, (6x) / 2)
   Evaluated: Dual2(1.201, 2.03, 0.30000000000000004) == Dual2(1.201, 2.03, 0.30000000000000004)

## 3. Structured Matrices

**Problem 3.1 (C)** Complete the implementation of `LowerTridiagonal` which represents a banded matrix with
bandwidths $(l,u) = (2,0)$ by storing only its diagonal, sub-diagonal, and second-sub-diagonal as vectors.

In [8]:
import Base: getindex,  size, *

struct LowerTridiagonal <: AbstractMatrix{Float64}
    d::Vector{Float64}   # diagonal entries of length n
    dl::Vector{Float64}  # sub-diagonal entries of length n-1
    dl2::Vector{Float64} # second-sub-diagonal entries of length n-2
end

size(L::LowerTridiagonal) = (length(L.d),length(L.d))

function getindex(L::LowerTridiagonal, k::Int, j::Int)
    d, dl, dl2 = L.d, L.dl, L.dl2
    # TODO: return L[k,j].
    # If `k == j` then it should be equal to `d[k]`.
    # If `k == j+1` then it should be equal to `dl[j]`.
    # If `k == j+2` then it should be equal to `dl2[j]`.
    # Otherwise, it should return 0.0
    ## SOLUTION
    if k == j
        d[k]
    elseif k == j+1
        dl[j]
    elseif k == j+2
        dl2[j]
    else
        0.0
    end
    ## END
end

n = 10
d, dl, dl2 = randn(n), randn(n-1), randn(n-2)
@test LowerTridiagonal(d, dl, dl2) == diagm(0 => d, -1 => dl, -2 => dl2)

Test Passed
  Expression: LowerTridiagonal(d, dl, dl2) == diagm(0 => d, -1 => dl, -2 => dl2)
   Evaluated: [-1.3060650083779026 0.0 … 0.0 0.0; -1.7190655458441395 -0.32549142470133213 … 0.0 0.0; … ; 0.0 0.0 … 0.7486471167904769 0.0; 0.0 0.0 … 0.11458311309250767 -1.6333698695008252] == [-1.3060650083779026 0.0 … 0.0 0.0; -1.7190655458441395 -0.32549142470133213 … 0.0 0.0; … ; 0.0 0.0 … 0.7486471167904769 0.0; 0.0 0.0 … 0.11458311309250767 -1.6333698695008252]

**Problem 3.2 (B)** Complete the implementation of `*` for a `LowerTridiagonal` matrix
so that it takes $O(n)$ operations.

In [9]:
function *(L::LowerTridiagonal, x::AbstractVector)
    ## TODO: Return L*x but computed in O(n) operations
    ## SOLUTION
    n,m = size(L)
    b = zeros(n) # returned vector

    for j = 1:n, k = j:min(j+2,n)
        b[k] += L[k,j]*x[j]
    end
    b
    ## END
end

n = 10
d, dl, dl2 = randn(n), randn(n-1), randn(n-2)
x = randn(n)
@test LowerTridiagonal(d, dl, dl2)*x ≈ diagm(0 => d, -1 => dl, -2 => dl2)*x

Test Passed
  Expression: LowerTridiagonal(d, dl, dl2) * x ≈ diagm(0 => d, -1 => dl, -2 => dl2) * x
   Evaluated: [1.0426642047679044, -1.805807111400234, -0.47593613900239584, 0.4063813641392797, 0.32988846510003056, 0.6728956798190676, -0.31723859650332453, 1.067084915281285, 1.178928708164844, 0.9993355798370727] ≈ [1.0426642047679044, -1.805807111400234, -0.4759361390023958, 0.4063813641392797, 0.32988846510003056, 0.6728956798190676, -0.31723859650332453, 1.0670849152812851, 1.178928708164844, 0.9993355798370727]

## 4. Decompositions

**Problem 4.1 (C)** Approximate $\exp x$ by a cubic polynomial by minimising
the least squares error when sampled at $n$ evenly spaced points in $[0,1]$,
that is, $x_k = (k-1)/(n-1)$,
returning the coefficients in the monomial basis.

In [10]:
function expfit(n)
    ## TODO: return the coefficients [c_0,c_1,c_2,c_3] of the polynomial
    # c_0 + c_1*x + c_2*x^2 + c_3*x^3 that minimises the L^2 error at `n`
    # evenly spaced samples
    ## SOLUTION
    x = range(0,1; length=n)
    V = x .^ (0:3)'
    V \ exp.(x)
    ## END
end

expfit (generic function with 1 method)

In [11]:
c₀,c₁,c₂,c₃ = expfit(1000)
    x = 0.1
    @test abs(c₀ + c₁*x + c₂*x^2 + c₃*x^3 - exp(x)) ≤ 1E-3

Test Passed
  Expression: abs((c₀ + c₁ * x + c₂ * x ^ 2 + c₃ * x ^ 3) - exp(x)) ≤ 0.001
   Evaluated: 0.00021335436497960103 ≤ 0.001

**Problem 4.2 (A)** Complete the function `lq(A)` that
returns a LQ decomposition, that is, `A = LQ` where  `L` is lower triangular and `Q` is an orthogonal
matrix. You may assume that `A`
is a square `Matrix{Float64}`. Hint: think of how a Householder reflection
can be constructed such that, for $𝐱 ∈ ℝ^n$,
$$
𝐱^⊤ Q = \|𝐱\|𝐞_1^⊤.
$$

In [12]:
function lq(A)
    m,n = size(A)
    m == n || error("not square")
    ## TODO Create Q and L such that A = L*Q, Q'Q == I and L is lower triangular
    ## SOLUTION
    L = copy(A)
    Q = Matrix(1.0I, n, n)
    for k = 1:n-1
        y = L[k, k:end]
        y[1] -= norm(y)
        w = y / norm(y)
        Qₖ = I - 2 * w * w'
        L[k:end, k:end] = L[k:end, k:end] * Qₖ
        Q[k:end, :] = Qₖ * Q[k:end, :]
    end
    L,Q
    ## END
end

lq (generic function with 1 method)

In [13]:
A = [1.0 2 3; 1 4 9; 1 1 1]
L,Q = lq(A)
@test Q'Q ≈ I
@test L*Q ≈ A
@test L ≈ tril(L) # it is acceptable to have small non-zero entries in L

Test Passed
  Expression: L ≈ tril(L)
   Evaluated: [3.7416573867739413 0.0 -1.1102230246251565e-16; 9.621404708847278 2.32992949004287 5.551115123125783e-17; 1.6035674514745462 -0.6131393394849659 0.22941573387056163] ≈ [3.7416573867739413 0.0 0.0; 9.621404708847278 2.32992949004287 0.0; 1.6035674514745462 -0.6131393394849659 0.22941573387056163]

## 5. Singular Value Decomposition

**Problem 5.1 (B)** Implement `pseudoinv` that returns the pseudo-inverse $A^+$
for an arbitrary square matrix, assuming that any singular value less than
$10^{-15}$ is in fact exactly zero. DO NOT use the inbuilt routine `pinv`.

In [14]:
function pseudoinv(A)
    m,n = size(A)
    m == n || error("A must be square")
    tol = 1E-15 # threshold below which we assume a singular value is zero
    ## TODO: construct and return the pseudo inverse of A
    ## SOLUTION
    U,σ,V = svd(A)
    r = 0
    for k = 1:length(σ)
        if σ[k] > tol
            r += 1
        end
    end
    V[:,1:r] * Diagonal(inv.(σ[1:r])) * U[:,1:r]'
    ## END
end

pseudoinv (generic function with 1 method)

In [15]:
A = [1 2 3; 4 5 6; 7 8 9]
A⁺ = pseudoinv(A)
@test A⁺*A*A⁺ ≈ A⁺
@test A*A⁺*A ≈ A

Test Passed
  Expression: A * A⁺ * A ≈ A
   Evaluated: [1.0000000000000018 2.0000000000000004 2.999999999999999; 4.000000000000003 5.000000000000001 6.0; 7.000000000000005 8.000000000000004 9.000000000000004] ≈ [1 2 3; 4 5 6; 7 8 9]

## 6. Differential Equations

**Problem 6.1 (B)** Complete the function `mathieu(n)` that returns a length-$n$ `Vector{Float64}`
$$
\begin{bmatrix}
u_1 \\
⋮ \\
u_n
\end{bmatrix}
$$
such that $u_k$ approximates the solution to the time-evolution equation
$$
\begin{align*}
u(0) &= 0 \\
u'(0) &= 1 \\
u''(t) &= cos(t) u(t)
\end{align*}
$$
at the point $t_k = (k-1)/(n-1)$ using the Forward Euler method, by first recasting the problem
as a vector ODE.

In [16]:
function mathieu(n)
    # TODO: return a Vector{Float64} approximating the solution to the ODE
    # at n evenly spaced points between 0 and 1.
    ## SOLUTION
    t = range(0, 1; length = n)
    A = t -> [0 1; cos(t) 0]
    h = step(t)

    U = zeros(2, n) # each column is a time-slice
    U[:, 1] = [0.0, 1.0] # initial condition
    for k = 1:n-1
        U[:, k+1] = (I + h * A(t[k])) * U[:, k]
    end

    U[1,:]
    ## END
end

mathieu (generic function with 1 method)

In [17]:
u = mathieu(100_000)
@test u isa Vector
@test abs(u[1]) ≤ 1E-15
# this compares against the exact formula
@test abs(u[end] - 1.148783704310448) ≤ 2E-5

Test Passed
  Expression: abs(u[end] - 1.148783704310448) ≤ 2.0e-5
   Evaluated: 4.409274623640158e-6 ≤ 2.0e-5